In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [2]:
# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

170498071/170498071 [==============================] - 3s 0us/step


In [4]:
# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [5]:
# Batch Normalization
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Lambda(tf.nn.local_response_normalization))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Lambda(tf.nn.local_response_normalization))

model.add(layers.Flatten())

In [6]:
# Dropouts
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10))

In [7]:
# Model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 lambda (Lambda)             (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 15, 15, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 lambda_1 (Lambda)           (None, 13, 13, 64)        0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                   

In [8]:
# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [9]:
# Fit the model with data augmentation
history = model.fit(datagen.flow(train_images, train_labels, batch_size=32),
                    steps_per_epoch=len(train_images) / 32,  # Adjust as needed
                    epochs=10,
                    validation_data=(test_images, test_labels))

Epoch 1/10
1562/1562 [==============================] - 137s 87ms/step - loss: 1.9911 - accuracy: 0.1069 - val_loss: 1.6609 - val_accuracy: 0.0913
Epoch 2/10
1562/1562 [==============================] - 147s 94ms/step - loss: 1.8305 - accuracy: 0.1000 - val_loss: 1.5744 - val_accuracy: 0.0858
Epoch 3/10
1562/1562 [==============================] - 140s 90ms/step - loss: 1.7523 - accuracy: 0.0957 - val_loss: 1.4395 - val_accuracy: 0.0522
Epoch 4/10
1562/1562 [==============================] - 135s 86ms/step - loss: 1.7034 - accuracy: 0.0916 - val_loss: 1.5112 - val_accuracy: 0.0594
Epoch 5/10
1562/1562 [==============================] - 133s 85ms/step - loss: 1.6834 - accuracy: 0.0920 - val_loss: 1.4899 - val_accuracy: 0.1023
Epoch 6/10
1562/1562 [==============================] - 141s 90ms/step - loss: 1.6443 - accuracy: 0.0916 - val_loss: 1.3881 - val_accuracy: 0.0783
Epoch 7/10
1562/1562 [==============================] - 136s 87ms/step - loss: 1.6277 - accuracy: 0.0926 - val_loss: 1

In [10]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print("Test accuracy:", test_acc)


313/313 - 6s - loss: 1.3139 - accuracy: 0.0829 - 6s/epoch - 19ms/step
Test accuracy: 0.08290000259876251
